# Lazy Predict for regression

In [4]:
#!pip install lazypredict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 11.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 10.2 MB/s eta 0:00:00a 0:00:01


In [1]:
# Imports
import tensorflow as tf
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
import numpy as np
import lazypredict
import matplotlib.pyplot as plt
import seaborn as sns
import random

#from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

2024-11-10 17:47:47.133001: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Set random seed for reproducibility

In [2]:
SEED = 64

#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Data Loading

In [3]:
# Only modify this
file_name = "DIS_lab_LoS_8"

In [4]:
df = pd.read_csv(f'../Datasets_benchmark/Regression/{file_name}.csv')

In [5]:
df.shape

(252004, 1602)

In [6]:
# Display the first few rows of the dataframe
print(df.head())

# Display the dataframe's information (column types, non-null values, etc.)
print(df.info())

# Get the number of rows and columns
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

   Antenna1Subcarrier1Module  Antenna1Subcarrier1Angle  \
0                       0.28                      3.14   
1                       0.31                      2.97   
2                       0.32                      2.75   
3                       0.34                      2.54   
4                       0.35                      2.49   

   Antenna1Subcarrier2Module  Antenna1Subcarrier2Angle  \
0                       0.27                      3.02   
1                       0.31                      2.83   
2                       0.32                      2.71   
3                       0.33                      2.55   
4                       0.34                      2.47   

   Antenna1Subcarrier3Module  Antenna1Subcarrier3Angle  \
0                       0.27                      3.05   
1                       0.30                      2.91   
2                       0.33                      2.72   
3                       0.36                      2.55   
4           

In [ ]:
# Summary statistics for numerical columns
print(df.describe())

# For categorical columns if any
#print(df.describe(include='object'))

In [ ]:
# Check for missing values
print(df.isnull().sum())

In [ ]:
# Histograms for numerical data
df.hist(bins=15, figsize=(15, 10))
plt.tight_layout()  # Adjusts subplot params so that subplots are nicely fit in the figure.
plt.show()

## Data Preprocessing

In [7]:
X = df.iloc[:,:-2]
X

,Antenna1Subcarrier1Module,Antenna1Subcarrier1Angle,Antenna1Subcarrier2Module,Antenna1Subcarrier2Angle,Antenna1Subcarrier3Module,Antenna1Subcarrier3Angle,Antenna1Subcarrier4Module,Antenna1Subcarrier4Angle,Antenna1Subcarrier5Module,Antenna1Subcarrier5Angle,...,Antenna8Subcarrier96Module,Antenna8Subcarrier96Angle,Antenna8Subcarrier97Module,Antenna8Subcarrier97Angle,Antenna8Subcarrier98Module,Antenna8Subcarrier98Angle,Antenna8Subcarrier99Module,Antenna8Subcarrier99Angle,Antenna8Subcarrier100Module,Antenna8Subcarrier100Angle
0,0.28,3.14,0.27,3.02,0.27,3.05,0.28,3.00,0.28,2.99,...,0.27,0.56,0.27,0.69,0.27,0.64,0.29,0.68,0.28,0.67
1,0.31,2.97,0.31,2.83,0.30,2.91,0.34,2.78,0.31,2.74,...,0.30,0.57,0.28,0.71,0.30,0.68,0.29,0.70,0.29,0.73
2,0.32,2.75,0.32,2.71,0.33,2.72,0.35,2.67,0.33,2.63,...,0.35,0.56,0.35,0.71,0.34,0.67,0.37,0.70,0.35,0.71
3,0.34,2.54,0.33,2.55,0.36,2.55,0.37,2.47,0.35,2.44,...,0.39,0.64,0.38,0.66,0.38,0.66,0.41,0.73,0.38,0.68
4,0.35,2.49,0.34,2.47,0.36,2.40,0.37,2.38,0.37,2.32,...,0.42,0.65,0.45,0.68,0.41,0.67,0.46,0.70,0.43,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251999,0.39,0.01,0.40,0.00,0.41,-0.06,0.42,-0.17,0.41,-0.20,...,0.55,-1.64,0.55,-1.68,0.56,-1.66,0.55,-1.62,0.55,-1.61
252000,0.33,-0.25,0.37,-0.21,0.39,-0.28,0.39,-0.39,0.41,-0.41,...,0.56,-1.83,0.54,-1.84,0.56,-1.88,0.56,-1.84,0.54,-1.84
252001,0.33,-0.45,0.35,-0.49,0.36,-0.59,0.37,-0.66,0.38,-0.66,...,0.56,-2.06,0.53,-2.10,0.56,-2.13,0.56,-2.09,0.55,-2.11
252002,0.32,-0.80,0.33,-0.86,0.33,-0.86,0.33,-0.98,0.35,-0.97,...,0.55,-2.36,0.54,-2.34,0.58,-2.34,0.54,-2.36,0.53,-2.36


In [8]:
y = df.iloc[:,-1]
y

0         1155
1         1160
2         1165
3         1170
4         1175
          ... 
251999    4009
252000    4014
252001    4019
252002    4024
252003    4029
Name: PositionY, Length: 252004, dtype: int64

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = SEED)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size = 0.50, random_state = SEED)

In [10]:
from sklearn.preprocessing import MinMaxScaler
# Create a MinMaxScaler object
scaler = MinMaxScaler()

In [11]:
# Scale numerical data
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_val = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)


## Model Architecture (Lazy Predict)

In [12]:
#lazypredict.Supervised.REGRESSORS = lazypredict.supervised.REGRESSORS[33:36]
lazypredict.Supervised.REGRESSORS

[('AdaBoostRegressor', sklearn.ensemble._weight_boosting.AdaBoostRegressor),
 ('BaggingRegressor', sklearn.ensemble._bagging.BaggingRegressor),
 ('BayesianRidge', sklearn.linear_model._bayes.BayesianRidge),
 ('DecisionTreeRegressor', sklearn.tree._classes.DecisionTreeRegressor),
 ('DummyRegressor', sklearn.dummy.DummyRegressor),
 ('ElasticNet', sklearn.linear_model._coordinate_descent.ElasticNet),
 ('ElasticNetCV', sklearn.linear_model._coordinate_descent.ElasticNetCV),
 ('ExtraTreeRegressor', sklearn.tree._classes.ExtraTreeRegressor),
 ('ExtraTreesRegressor', sklearn.ensemble._forest.ExtraTreesRegressor),
 ('GammaRegressor', sklearn.linear_model._glm.glm.GammaRegressor),
 ('GaussianProcessRegressor',
  sklearn.gaussian_process._gpr.GaussianProcessRegressor),
 ('GradientBoostingRegressor', sklearn.ensemble._gb.GradientBoostingRegressor),
 ('HistGradientBoostingRegressor',
  sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor),
 ('HuberRegressor', sk

In [ ]:
# Exclude specific regressors for the MIMO dataset due to its large size
# Define indices of regressors to be excluded
#excluded_indices = {10, 15, 26, 31, 32, 33, 37}
#
## Update the REGRESSORS list by keeping elements that are not in the excluded indices
#lazypredict.Supervised.REGRESSORS = [
#    model for i, model in enumerate(lazypredict.Supervised.REGRESSORS) if i not in excluded_indices
#]

In [14]:
lazypredict.Supervised.REGRESSORS

[('AdaBoostRegressor', sklearn.ensemble._weight_boosting.AdaBoostRegressor),
 ('BaggingRegressor', sklearn.ensemble._bagging.BaggingRegressor),
 ('BayesianRidge', sklearn.linear_model._bayes.BayesianRidge),
 ('DecisionTreeRegressor', sklearn.tree._classes.DecisionTreeRegressor),
 ('DummyRegressor', sklearn.dummy.DummyRegressor),
 ('ElasticNet', sklearn.linear_model._coordinate_descent.ElasticNet),
 ('ElasticNetCV', sklearn.linear_model._coordinate_descent.ElasticNetCV),
 ('ExtraTreeRegressor', sklearn.tree._classes.ExtraTreeRegressor),
 ('ExtraTreesRegressor', sklearn.ensemble._forest.ExtraTreesRegressor),
 ('GammaRegressor', sklearn.linear_model._glm.glm.GammaRegressor),
 ('GradientBoostingRegressor', sklearn.ensemble._gb.GradientBoostingRegressor),
 ('HistGradientBoostingRegressor',
  sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor),
 ('HuberRegressor', sklearn.linear_model._huber.HuberRegressor),
 ('KNeighborsRegressor', sklearn.neighbors._re

In [15]:
clf = LazyRegressor(verbose=2, ignore_warnings=True, custom_metric=None)

## Model Training

In [16]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score

In [17]:
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

  3%|▎         | 1/35 [51:11<29:00:21, 3071.22s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.5356563022991652, 'Adjusted R-Squared': 0.5041753736414814, 'RMSE': 603.307111119831, 'Time taken': 3071.2200407981873}


  6%|▌         | 2/35 [2:04:49<35:25:04, 3863.77s/it]

{'Model': 'BaggingRegressor', 'R-Squared': 0.8971241160377775, 'Adjusted R-Squared': 0.8901494798369489, 'RMSE': 283.9719584046848, 'Time taken': 4418.545289516449}


  9%|▊         | 3/35 [2:05:24<18:48:15, 2115.47s/it]

...

{'Model': 'TweedieRegressor', 'R-Squared': 0.6178249323911105, 'Adjusted R-Squared': 0.5919147583159314, 'RMSE': 547.3307422929265, 'Time taken': 20.98958683013916}


 97%|█████████▋| 34/35 [8:11:02<01:50, 110.75s/it]

{'Model': 'XGBRegressor', 'R-Squared': 0.9033320546150208, 'Adjusted R-Squared': 0.8967782956058696, 'RMSE': 275.2706890475414, 'Time taken': 45.180272579193115}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.631167 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 408000
[LightGBM] [Info] Number of data points in the train set: 201603, number of used features: 1600
[LightGBM] [Info] Start training from score 2592.445405


100%|██████████| 35/35 [8:11:41<00:00, 842.90s/it]

{'Model': 'LGBMRegressor', 'R-Squared': 0.8357662978216989, 'Adjusted R-Squared': 0.8246318095384242, 'RMSE': 358.79783023045417, 'Time taken': 39.11084794998169}


## Model Evaluation

In [18]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
KNeighborsRegressor,1.00,1.00,20.72,24.66
ExtraTreesRegressor,0.95,0.96,187.60,9710.27
MLPRegressor,0.92,0.92,243.63,1598.55
XGBRegressor,0.90,0.90,275.27,45.18
BaggingRegressor,0.89,0.90,283.97,4418.55
HistGradientBoostingRegressor,0.82,0.84,358.67,53.75
LGBMRegressor,0.82,0.84,358.80,39.11
ExtraTreeRegressor,0.76,0.78,419.25,108.95
BayesianRidge,0.74,0.76,433.20,35.00


In [19]:
# Convert results into a DataFrame
test_results = pd.DataFrame(models)

In [20]:
# Create the 'model_results' directory if it doesn't exist
if not os.path.exists('model_results'):
    os.mkdir('model_results')

# Save to a text file
test_results.to_csv(f'model_results/{file_name}/{file_name}_results_test_Y.txt', index=True)